In [ ]:
%reload_ext autoreload
%autoreload 2

import numpy as np
import sys
import os
import pickle
import argparse
import math
import time
from bisect import bisect_left
import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torchvision.transforms as trn
import torchvision.datasets as dset
import torch.nn.functional as F
from torch.autograd import Variable as V
import torchtext

import pandas as pd
from torchtext import data
from torchtext import datasets
import spacy
import re

import csv
csv.field_size_limit(sys.maxsize)

import tqdm
from tqdm import tqdm_notebook

In [ ]:
np.random.seed(1)

args = argparse.Namespace(
    batch_size = 64,
    in_dist_dataset = '20ng',
    method='OECC',
    save = 'results',
    load = 'results', 
    oe_dataset = 'wikitext2'
    )

torch.set_grad_enabled(False)
cudnn.benchmark = True  # fire on all cylinders

In [ ]:
from utils.display_results import get_performance

In [ ]:
# ============================ 20 Newsgroups ============================ #
TEXT_20ng = data.Field(pad_first=True, lower=True, fix_length=100)
LABEL_20ng = data.Field(sequential=False)

train_20ng = data.TabularDataset(path='20ng-train.txt',
                                 format='csv',
                                 fields=[('label', LABEL_20ng),('text', TEXT_20ng)])

test_20ng = data.TabularDataset(path='20ng-test.txt',
                                 format='csv',
                                 fields=[('label', LABEL_20ng),('text', TEXT_20ng)])

TEXT_20ng.build_vocab(train_20ng, max_size=10000)
LABEL_20ng.build_vocab(train_20ng, max_size=10000)
print('vocab length (including special tokens):', len(TEXT_20ng.vocab))
num_classes = len(LABEL_20ng.vocab)
print('num labels:', len(LABEL_20ng.vocab))
train_iter_20ng = data.BucketIterator(train_20ng, batch_size=args.batch_size, repeat=False)
test_iter_20ng = data.BucketIterator(test_20ng, batch_size=args.batch_size, repeat=False)

# ============================ 20 Newsgroups ============================ #

ood_num_examples = len(test_iter_20ng.dataset) // 5
expected_ap = ood_num_examples / (ood_num_examples + len(test_iter_20ng.dataset))
recall_level = 0.9

In [ ]:
# ============================ IMDB ============================ #

# set up fields
TEXT_imdb = data.Field(pad_first=True, lower=True)
LABEL_imdb = data.Field(sequential=False)

# make splits for data
train_imdb, test_imdb = datasets.IMDB.splits(TEXT_imdb, LABEL_imdb)

# build vocab
TEXT_imdb.build_vocab(train_20ng.text, max_size=10000)
LABEL_imdb.build_vocab(train_imdb, max_size=10000)
print('vocab length for IMBD (including special tokens):', len(TEXT_imdb.vocab))

# make iterators
train_iter_imdb, test_iter_imdb = data.BucketIterator.splits(
    (train_imdb, test_imdb), batch_size=args.batch_size, repeat=False)

# ============================ IMDB ============================ #

# ============================ SNLI ============================ #

# set up fields
TEXT_snli = data.Field(pad_first=True, lower=True)
LABEL_snli = data.Field(sequential=False)

# make splits for data
train_snli, val_snli, test_snli = datasets.SNLI.splits(TEXT_snli, LABEL_snli)

# build vocab
TEXT_snli.build_vocab(train_20ng.text, max_size=10000)
LABEL_snli.build_vocab(train_snli, max_size=10000)
print('vocab length for SNLI (including special tokens):', len(TEXT_snli.vocab))

# make iterators
train_iter_snli, val_iter_snli, test_iter_snli = data.BucketIterator.splits(
    (train_snli, val_snli, test_snli), batch_size=args.batch_size, repeat=False)

# ============================ SNLI ============================ #

# ============================ Multi30K ============================ #
TEXT_m30k = data.Field(pad_first=True, lower=True)
LABEL_m30k = data.Field(pad_first=True, lower=True)

data_m30k = datasets.Multi30k.splits(('.en', '.de'),(TEXT_m30k, LABEL_m30k), path='./.data/multi30k/',train=None, validation=None)

TEXT_m30k.build_vocab(train_20ng.text, max_size=10000)
LABEL_m30k.build_vocab(data_m30k[0], max_size=10000)
print('vocab length for Multi30k (including special tokens):', len(TEXT_m30k.vocab))

train_iter_m30k = data.BucketIterator(data_m30k[0], batch_size=args.batch_size, repeat=False)
# ============================ Multi30K ============================ #

# ============================ WMT16 ============================ #
TEXT_wmt16 = data.Field(pad_first=True, lower=True)
LABEL_wmt16 = data.Field(pad_first=True, lower=True)

data_wmt16 = datasets.TranslationDataset.splits(('.en', '.de'),(TEXT_wmt16, LABEL_wmt16), path='./.data/wmt16/',train=None, validation=None)

TEXT_wmt16.build_vocab(train_20ng.text, max_size=10000)
LABEL_wmt16.build_vocab(data_wmt16[0], max_size=10000)
print('vocab length for WMT16 (including special tokens):', len(TEXT_wmt16.vocab))

train_iter_wmt16 = data.BucketIterator(data_wmt16[0], batch_size=args.batch_size, repeat=False)
# ============================ WMT16 ============================ #

# ============================ English Web Treebank (Answers) ============================ #

TEXT_answers = data.Field(pad_first=True, lower=True)

treebank_path = './.data/eng_web_tbk/answers-dev.conllu'

train_answers = datasets.SequenceTaggingDataset(path=treebank_path, fields=((None, None), ('text', TEXT_answers)))

TEXT_answers.build_vocab(train_20ng.text, max_size=10000)
print('vocab length for EWT-Answers (including special tokens):', len(TEXT_answers.vocab))

# make iterators
train_iter_answers = data.BucketIterator.splits(
    (train_answers,), batch_size=args.batch_size, repeat=False)[0]

# ============================ English Web Treebank (Answers) ============================ #

# ============================ English Web Treebank (Email) ============================ #

TEXT_email = data.Field(pad_first=True, lower=True)

treebank_path = './.data/eng_web_tbk/email-dev.conllu'

train_email = datasets.SequenceTaggingDataset(path=treebank_path, fields=((None, None), ('text', TEXT_email)))

TEXT_email.build_vocab(train_20ng.text, max_size=10000)
print('vocab length for EWT-Email (including special tokens):', len(TEXT_email.vocab))

# make iterators
train_iter_email = data.BucketIterator.splits(
    (train_email,), batch_size=args.batch_size, repeat=False)[0]

# ============================ English Web Treebank (Email) ============================ #

# ============================ English Web Treebank (Newsgroup) ============================ #

TEXT_newsgroup = data.Field(pad_first=True, lower=True)

treebank_path = './.data/eng_web_tbk/newsgroup-dev.conllu'

train_newsgroup = datasets.SequenceTaggingDataset(path=treebank_path, fields=((None, None), ('text', TEXT_newsgroup)))

TEXT_newsgroup.build_vocab(train_20ng.text, max_size=10000)
print('vocab length for EWT-Newsgroup (including special tokens):', len(TEXT_newsgroup.vocab))

# make iterators
train_iter_newsgroup = data.BucketIterator.splits(
    (train_newsgroup,), batch_size=args.batch_size, repeat=False)[0]

# ============================ English Web Treebank (Newsgroup) ============================ #

# ============================ English Web Treebank (Reviews) ============================ #

TEXT_reviews = data.Field(pad_first=True, lower=True)

treebank_path = './.data/eng_web_tbk/reviews-dev.conllu'

train_reviews = datasets.SequenceTaggingDataset(path=treebank_path, fields=((None, None), ('text', TEXT_reviews)))

TEXT_reviews.build_vocab(train_20ng.text, max_size=10000)
print('vocab length for EWT-Reviews (including special tokens):', len(TEXT_reviews.vocab))

# make iterators
train_iter_reviews = data.BucketIterator.splits(
    (train_reviews,), batch_size=args.batch_size, repeat=False)[0]

# ============================ English Web Treebank (Reviews) ============================ #

# ============================ English Web Treebank (Weblog) ============================ #

TEXT_weblog = data.Field(pad_first=True, lower=True)

treebank_path = './.data/eng_web_tbk/reviews-dev.conllu'

train_weblog = datasets.SequenceTaggingDataset(path=treebank_path, fields=((None, None), ('text', TEXT_weblog)))

TEXT_weblog.build_vocab(train_20ng.text, max_size=10000)
print('vocab length for EWT-Weblog (including special tokens):', len(TEXT_weblog.vocab))

# make iterators
train_iter_weblog = data.BucketIterator.splits(
    (train_weblog,), batch_size=args.batch_size, repeat=False)[0]

# ============================ English Web Treebank (Weblog) ============================ #

# ============================ Yelp Reviews ============================ #
TEXT_yelp = data.Field(pad_first=True, lower=True)

yelp_data = data.TabularDataset(path='./yelp.csv',
                                  format='csv',
                                  fields=[(None, None), ('text', TEXT_yelp)],skip_header=True)

TEXT_yelp.build_vocab(train_20ng.text, max_size=10000)
print('vocab length for Yelp Reviews (including special tokens):', len(TEXT_yelp.vocab))

train_iter_yelp = data.BucketIterator(yelp_data, batch_size=args.batch_size, repeat=False)
# ============================ Yelp Reviews ============================ #

In [ ]:
class ClfGRU(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.embedding = nn.Embedding(len(TEXT_20ng.vocab), 50, padding_idx=1)
        self.gru = nn.GRU(input_size=50, hidden_size=128, num_layers=2, bias=True, batch_first=True, bidirectional=False)
        self.linear = nn.Linear(128, num_classes)
        self.num_classes = num_classes

    def forward(self, x):
        embeds = self.embedding(x)
        hidden = self.gru(embeds)[1][1]  # select h_n, and select the 2nd layer
        logits = self.linear(hidden)
        return logits



model = ClfGRU(num_classes-1)
model.load_state_dict(torch.load(f'./{args.load}/{args.in_dist_dataset}/{args.method}/{args.oe_dataset}/model_finetune.dict'))  # change location as per our method
print('\nLoaded model.\n')


def get_scores(dataset_iterator, ood=False, translation_dataset = False, snli=False):
    model.eval()
    model.cpu()
    
    outlier_scores = []

    for batch_idx, batch in enumerate(iter(dataset_iterator)):
        if ood and (batch_idx * args.batch_size > ood_num_examples):
            break

        if snli:
            inputs = batch.hypothesis.t()
        else:
            if translation_dataset:
                inputs = batch.src.t()
            else:        
                inputs = batch.text.t()

        logits = model(inputs)
        smax = F.softmax(logits - torch.max(logits, dim=1, keepdim=True)[0], dim=1)
        msp = -1 * torch.max(smax, dim=1)[0]

        # ce_to_unif = F.log_softmax(logits - torch.max(logits, dim=1, keepdim=True)[0], dim=1).mean(1)

        outlier_scores.extend(list(msp.data.cpu().numpy()))

    return outlier_scores



# ============================ OECC ============================ #

test_scores = get_scores(test_iter_20ng)

titles = ['SNLI', 'IMDB', 'Multi30K', 'WMT16', 'English Web Treebank (Answers)',
          'English Web Treebank (Email)', 'English Web Treebank (Newsgroup)',
          'English Web Treebank (Reviews)', 'English Web Treebank (Weblog)',
          'Yelp Reviews']

iterators = [test_iter_snli, test_iter_imdb, train_iter_m30k, train_iter_wmt16, train_iter_answers,
             train_iter_email, train_iter_newsgroup, train_iter_reviews, train_iter_weblog,
             train_iter_yelp]


mean_fprs = []
mean_aurocs = []
mean_auprs = []

f = open(f'./{args.save}/{args.in_dist_dataset}/{args.method}/{args.oe_dataset}/OECC_eval_results.txt', 'w')

for i in range(len(titles)):
    title = titles[i]
    iterator = iterators[i]
    
    if '30K' in title or '16' in title:
        translation_dataset=True
    else:
        translation_dataset=False

    print(f'\n{title}')
    f.write(f'\n{title}')
    fprs, aurocs, auprs = [], [], []
    for i in range(10):
        ood_scores = get_scores(iterator, ood=True, translation_dataset = translation_dataset, snli=True) if 'SNLI' in title else get_scores(iterator, ood=True, translation_dataset=translation_dataset)
        fpr, auroc, aupr = get_performance(ood_scores, test_scores, expected_ap, recall_level=recall_level)
        fprs.append(fpr)
        aurocs.append(auroc)
        auprs.append(aupr)

    print(f'FPR{int(100 * recall_level):d}:\t\t\t{np.mean(fprs):.4f} ({np.std(fprs):.4f})')
    f.write(f'\nFPR{int(100 * recall_level):d}:\t\t\t{np.mean(fprs):.4f} ({np.std(fprs):.4f})')
    print(f'AUROC:\t\t\t{np.mean(aurocs):.4f} ({np.std(aurocs):.4f})')
    f.write(f'\nAUROC:\t\t\t{np.mean(aurocs):.4f} ({np.std(aurocs):.4f})')
    print(f'AUPR:\t\t\t{np.mean(auprs):.4f} ({np.std(auprs):.4f})')
    f.write(f'\nAUPR:\t\t\t{np.mean(auprs):.4f} ({np.std(auprs):.4f})\n')

    mean_fprs.append(np.mean(fprs))
    mean_aurocs.append(np.mean(aurocs))
    mean_auprs.append(np.mean(auprs))

print()
print(f'OOD dataset mean FPR: {np.mean(mean_fprs):.4f}')
f.write(f'\nOOD dataset mean FPR: {np.mean(mean_fprs):.4f}')
print(f'OOD dataset mean AUROC: {np.mean(mean_aurocs):.4f}')
f.write(f'\nOOD dataset mean AUROC: {np.mean(mean_aurocs):.4f}')
print(f'OOD dataset mean AUPR: {np.mean(mean_auprs):.4f}')
f.write(f'\nOOD dataset mean AUPR: {np.mean(mean_auprs):.4f}')

f.close()